In [1]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
from typing import List
import pandas as pd

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,3 hours 32 mins
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.1
H2O_cluster_version_age:,"21 days, 3 hours and 48 minutes"
H2O_cluster_name:,H2O_from_python_thomas_d49f2k
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.599 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [7]:
#h2o.remove_all()
h2o.ls()

,key
0,Key_Frame__test_subset.hex
1,Key_Frame__train_subset.hex
2,Key_Frame__validation_subset.hex
3,combined-prediction-363710cd-66d8-4128-b194-b3...
4,gbm-02bf2457-a731-415f-b898-49aaf7700c3e
5,modelmetrics_gbm-02bf2457-a731-415f-b898-49aaf...
6,modelmetrics_gbm-02bf2457-a731-415f-b898-49aaf...
7,modelmetrics_gbm-02bf2457-a731-415f-b898-49aaf...
8,prediction-363710cd-66d8-4128-b194-b30484bf588c


In [3]:
random_state = 1
input_features = ['County', 'State', 'Area', 'VicAge', 'VicSex', 'VicRace', 'VicEthnic', 'VicCount', 'Weapon', 'Subcircum', 'Agency', 'Agentype', 'Circumstance', 'Homicide']
output_features = ['OffAge', 'OffSex', 'OffRace', 'OffEthnic', 'OffCount', 'Relationship']

def load_subsets() -> List[pd.DataFrame]:
    # loads the seperately saved train/test/validation subsets instead of loading, cleaning and sampling data from scraftch every time
    # use of this function should be mutualy exclusive with method "prepare_data"
    train_sample = h2o.import_file('train_subset.csv', destination_frame = 'train_sample')
    test_sample = h2o.import_file('test_subset.csv', destination_frame = 'test_sample')
    validation_sample = h2o.import_file('validation_subset.csv', destination_frame = 'validation_sample')

    return train_sample, test_sample, validation_sample

train_sample, test_sample, validation_sample = load_subsets()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
list = h2o.ls()
print(list)
combined = list[3]
combined.head()

                                                 key
0                         Key_Frame__test_subset.hex
1                        Key_Frame__train_subset.hex
2                   Key_Frame__validation_subset.hex
3  combined-prediction-363710cd-66d8-4128-b194-b3...
4           gbm-02bf2457-a731-415f-b898-49aaf7700c3e
5  modelmetrics_gbm-02bf2457-a731-415f-b898-49aaf...
6  modelmetrics_gbm-02bf2457-a731-415f-b898-49aaf...
7  modelmetrics_gbm-02bf2457-a731-415f-b898-49aaf...
8    prediction-363710cd-66d8-4128-b194-b30484bf588c
9                                      py_3_sid_86b5


KeyError: 3

In [18]:
hyper_parameters_1 = {'max_depth': [5, 7, 10], 'min_rows': [5, 10, 20, 30, 40, 50, 60, 100]}
one_tree_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator, grid_id='one_tree_grid_1', hyper_params=hyper_parameters_1)
one_tree_grid1.train(x=input_features, y='OffAge', training_frame=train_sample, validation_frame=validation_sample, seed=random_state)

result_grid_1= one_tree_grid1.get_grid()
result_grid_1

# additional hyper-parameters: 'min_rows': [5, 10, 20, 30, 40, 50, 60, 100], 'min_split_improvement': [1e-3, 1e-5, 1e-7, 1e-10]

gbm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%
      max_depth  min_rows  min_split_improvement                 model_ids  \
0           5.0       5.0                0.00100   one_tree_grid_1_model_1   
1           5.0      20.0                0.00001  one_tree_grid_1_model_15   
2           5.0      60.0                0.00001  one_tree_grid_1_model_27   
3           5.0      50.0                0.00001  one_tree_grid_1_model_24   
4           5.0      10.0                0.00001   one_tree_grid_1_model_4   
5           5.0     100.0                0.00001  one_tree_grid_1_model_30   
6           5.0      30.0                0.00001  one_tree_grid_1_model_18   
7           5.0      40.0                0.00001  one_tree_grid_1_model_21   
8           5.0       5.0                0.00001   one_tree_grid_1_model_9   
9           7.0       5.0                0.00100   one_tree_grid_1_model_2   
10          7.0     100.0                0.0000

In [19]:
train_sample.head()

C1,Area,County,FileMonth,FileDay,FileYear,State,Agency,Agentype,Source,Solved,Year,Month,ActionType,Homicide,VicAge,VicSex,VicRace,VicEthnic,OffAge,OffSex,OffRace,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount
0,Atlanta-Sandy Springs-Marietta,Fulton,3,21,11,Georgia,Sandy Spring,Municipal police,FBI,Yes,2010,September,Normal update,Murder and non-negligent manslaughter,2,Female,Black,Not of Hispanic origin,0-11,Female,Black,Not of Hispanic origin,Unknown,Other - known to victim,Child killed by babysitter,Unknown,0,0
1,Columbus,Franklin,4,5,83,Ohio,Columbus,Municipal police,FBI,Yes,1982,October,Adjustment,Murder and non-negligent manslaughter,14,Male,Black,Not of Hispanic origin,0-11,Female,Black,Not of Hispanic origin,Knife or cutting instrument,Stranger,Other arguments,Unknown,0,0
2,Philadelphia-Camden-Wilmington,Philadelphia,1,18,95,Pennsylvania,Philadelphia,Municipal police,FBI,Yes,1994,July,Normal update,Murder and non-negligent manslaughter,0,Female,Black,Unknown,0-11,Female,Black,Unknown,"Personal weapons, includes beating",Other family,Other,Unknown,0,0
3,Minneapolis-St. Paul-Bloomington,Hennepin,4,1,81,Minnesota,Minneapolis,Municipal police,FBI,Yes,1980,November,Adjustment,Murder and non-negligent manslaughter,0,Female,Black,Unknown,0-11,Female,Black,Unknown,Unknown,Acquaintance,Child killed by babysitter,Unknown,0,0
4,Denver-Aurora,Denver,7,13,1,Colorado,Denver,Municipal police,FBI,Yes,2001,March,Normal update,Manslaughter by negligence,46,Male,Black,Unknown,0-11,Female,Black,Unknown,Fire,Acquaintance,All other manslaughter by negligence,Unknown,0,0
5,Rockford,Winnebago,3,16,95,Illinois,Rockford,Municipal police,FBI,Yes,1994,August,Normal update,Murder and non-negligent manslaughter,0,Male,Black,Unknown,0-11,Female,Black,Unknown,"Blunt object - hammer, club, etc",Other family,Other,Unknown,0,0
6,New Orleans-Metairie-Kenner,Orleans,2,15,94,Louisiana,New Orleans,Municipal police,FBI,Yes,1993,September,Normal update,Murder and non-negligent manslaughter,0,Male,Black,Unknown,0-11,Female,Black,Unknown,"Personal weapons, includes beating",Other family,Other,Unknown,0,0
7,Chicago-Naperville-Joliet,Cook,3,1,80,Illinois,Chicago,Municipal police,FBI,Yes,1979,May,Adjustment,Manslaughter by negligence,1,Male,Black,Unknown,0-11,Female,Black,Unknown,Asphyxiation - includes death by gas,Neighbor,All other manslaughter by negligence,Unknown,0,0
8,St. Louis,St. Louis,6,5,86,Missouri,St. Louis County Police Department,County police,FBI,Yes,1986,March,Normal update,Murder and non-negligent manslaughter,10,Male,White,Not of Hispanic origin,0-11,Female,White,Not of Hispanic origin,"Handgun - pistol, revolver, etc",Acquaintance,Other arguments,Unknown,0,0
9,Fort Wayne,Allen,8,30,1,Indiana,Fort Wayne,Municipal police,FBI,Yes,2001,February,Normal update,Manslaughter by negligence,4,Male,Black,Unknown,0-11,Female,White,Unknown,"Firearm, type not stated",Other family,Children playing with gun,Unknown,0,0


## Building the multi-output Tree

In [18]:
one_tree = H2OGradientBoostingEstimator(ntrees=1, max_depth=5, seed = random_state)
one_tree.train(x = input_features, y = 'OffAge', training_frame = train_sample)
# one_tree.model_performance(test_sample).accuracy()
predictions = one_tree.predict(test_sample)
print(predictions)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/Users/thomas/opt/anaconda3/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'County' has levels not trained on: ["Ballard", "Banks", "Box Butte", "Chowan", "Cimarron", "Clear Creek", "Day", "Doniphan", "Elliott", "Ellsworth", ...25 not listed..., "Schley", "Stonewall", "Teton", "Towner", "Trempealeau", "Wallowa", "Walsh", "Warrick", "Washakie", "Yellow Medicine"]
  warnings.warn(w)
/Users/thomas/opt/anaconda3/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'Agency' has levels not trained on: ["Acton", "Adairsville", "Alamo Community College District", "Alamosa", "Albia", "Allegany State Police", "Allegheny Township", "Antlers", "Asotin County", "Audubon", ...562 not listed..., "Windham Town", "Winfield Township", "Winston", "Winston-Salem State Univ", "Wood-Ridge", "Woodbridge", "Worland", "Yale University", "Yellow Medicine County", "Yorktown"]
  warnings.warn(w)


predict,0-11,12-14,15-17,18-21,22-24,25-29,30-34,35-39,40-49,50-64,65+,Unknown
12-14,0.0671881,0.223072,0.0697152,0.0682007,0.0737514,0.073916,0.0772392,0.068193,0.069548,0.0702253,0.0674731,0.0714778
18-21,0.0754351,0.0760381,0.0786543,0.116086,0.0885761,0.0807495,0.085567,0.0779921,0.0760669,0.0752653,0.074748,0.0948214
30-34,0.077261,0.0778837,0.0752731,0.0903915,0.0814263,0.0814712,0.121755,0.0806971,0.0800199,0.0784923,0.0764178,0.0789113
50-64,0.0738957,0.0738886,0.0747253,0.0793601,0.0771018,0.0739626,0.0820706,0.0825838,0.116158,0.118123,0.0739983,0.0741321
15-17,0.0752141,0.0768369,0.110545,0.0905319,0.0912759,0.0827457,0.0791536,0.086296,0.0803397,0.0760559,0.0755331,0.0754719
12-14,0.0683174,0.226822,0.0684772,0.069347,0.0770146,0.0751584,0.0694656,0.0693393,0.0698178,0.069082,0.0686072,0.0685516
12-14,0.0665955,0.221105,0.0794016,0.0675992,0.0750734,0.0840258,0.0677147,0.0675916,0.0689346,0.0673408,0.066878,0.0677399
25-29,0.074995,0.0754099,0.077575,0.0862205,0.0902491,0.108447,0.0938695,0.0805686,0.0800874,0.0820474,0.0752959,0.0752349
35-39,0.0744553,0.0744553,0.0772558,0.0896186,0.0896201,0.0819109,0.0994311,0.101742,0.0795292,0.0814755,0.0747711,0.0757348
0-11,0.123784,0.105704,0.0998457,0.0893325,0.0765825,0.0747368,0.0714924,0.0757049,0.0694261,0.0686945,0.0682224,0.0764736


In [26]:
#feature_interactions = one_tree.feature_interaction()
#print(feature_interactions)

predictions2 = h2o.predict

predict,0-11,12-14,15-17,18-21,22-24,25-29,30-34,35-39,40-49,50-64,65+,Unknown
12-14,0.0671881,0.223072,0.0697152,0.0682007,0.0737514,0.073916,0.0772392,0.068193,0.069548,0.0702253,0.0674731,0.0714778
18-21,0.0754351,0.0760381,0.0786543,0.116086,0.0885761,0.0807495,0.085567,0.0779921,0.0760669,0.0752653,0.074748,0.0948214
30-34,0.077261,0.0778837,0.0752731,0.0903915,0.0814263,0.0814712,0.121755,0.0806971,0.0800199,0.0784923,0.0764178,0.0789113
50-64,0.0738957,0.0738886,0.0747253,0.0793601,0.0771018,0.0739626,0.0820706,0.0825838,0.116158,0.118123,0.0739983,0.0741321
15-17,0.0752141,0.0768369,0.110545,0.0905319,0.0912759,0.0827457,0.0791536,0.086296,0.0803397,0.0760559,0.0755331,0.0754719
12-14,0.0683174,0.226822,0.0684772,0.069347,0.0770146,0.0751584,0.0694656,0.0693393,0.0698178,0.069082,0.0686072,0.0685516
12-14,0.0665955,0.221105,0.0794016,0.0675992,0.0750734,0.0840258,0.0677147,0.0675916,0.0689346,0.0673408,0.066878,0.0677399
25-29,0.074995,0.0754099,0.077575,0.0862205,0.0902491,0.108447,0.0938695,0.0805686,0.0800874,0.0820474,0.0752959,0.0752349
35-39,0.0744553,0.0744553,0.0772558,0.0896186,0.0896201,0.0819109,0.0994311,0.101742,0.0795292,0.0814755,0.0747711,0.0757348
0-11,0.123784,0.105704,0.0998457,0.0893325,0.0765825,0.0747368,0.0714924,0.0757049,0.0694261,0.0686945,0.0682224,0.0764736
